## `property()`

`properties` allow you to create methods that behave like attributes.

If you ever need to change the underlying implementation, then you can turn the attribute into a property at any time without much pain.

In [ ]:
property(fget=None, fset=None, fdel=None, doc=None)

<div class="alert alert-success" role="alert">
    Properties are <strong>class attributes</strong> that manage <strong>instance attributes</strong>.
</div>

Here’s a recap of some important points to remember when you’re creating properties with the decorator approach:

- The `@property` decorator must decorate the **getter method**.
- The docstring must go in the **getter method**.
- The **setter and deleter** methods must be decorated with the name of the getter method plus `.setter` and `.deleter`, respectively.

---

## `property()` as a function

In [2]:
class Car:
    def __init__(self, speed):
        self.speed = speed
        
    def get_speed(self):
        print('Calling get...')
        return self._speed
    
    def set_speed(self, value):
        print('Calling set...')
        if value < 0:
            raise ValueError('speed must be greater than 0')
        self._speed = value
        
    def del_speed(self):
        del self._speed
        
    speed = property(get_speed, set_speed, del_speed)
        

In [3]:
print("Creating an object:")
c = Car(10)
print("\nGet the value of c.speed:")
print(c.speed)
print("\nGet the value of c._speed:")
print(c._speed)
print("\nIs _speed an instance attibute:", "_speed" in c.__dict__)
print("Is _speed an class attibute   :", "_speed" in Car.__dict__)
print("\nIs  speed an instance attibute:", "speed" in c.__dict__)
print("Is  speed an class attibute   :", "speed" in Car.__dict__)

Creating an object:

Get the value of c.speed:
Calling get...
10

Get the value of c._speed:
10

Is _speed an instance attibute: True
Is _speed an class attibute   : False

Is  speed an instance attibute: False
Is  speed an class attibute   : True


---

## `property()` as a decorator

In [5]:
class Car:
    def __init__(self, speed):
        self.speed = speed
    
    @property
    def speed(self):
        print('Calling get...')
        return self._speed
    
    @speed.setter
    def speed(self, value):
        print('Calling set...')
        if value < 0:
            raise ValueError('speed must be greater than 0')
        self._speed = value
        
    @speed.deleter
    def speed(self):
        print('Calling del...')
        del self._speed

In [8]:
print("Creating an object:")
c = Car(10)
print("\nGet the value of c.speed:")
print(c.speed)
print("\nGet the value of c._speed:")
print(c._speed)
print("\nIs _speed an instance attibute:", "_speed" in c.__dict__)
print("Is _speed an class attibute   :", "_speed" in Car.__dict__)
print("\nIs  speed an instance attibute:", "speed" in c.__dict__)
print("Is  speed an class attibute   :", "speed" in Car.__dict__)

Creating an object:
Calling set...

Get the value of c.speed:
Calling get...
10

Get the value of c._speed:
10

Is _speed an instance attibute: True
Is _speed an class attibute   : False

Is  speed an instance attibute: False
Is  speed an class attibute   : True


---

## Read-Only Attributes

In [10]:
class WriteSpeedError(Exception):
    pass

class Car:
    def __init__(self, speed):
        print('Calling init...')
        self._speed = speed     # Because speed is raise error when setting value, we use _speed here.
    
    @property
    def speed(self):
        print('Calling get...')
        return self._speed
    
    @speed.setter
    def speed(self, value):
        print('Calling set...')
        raise WriteSpeedError('speed attribute is read-only')

In [12]:
print("Creating an object:")
c = Car(50)
print("\nGet the value of c.speed:")
print(c.speed)
print("\nSet the value of c.speed:")
c.speed = 100

Creating an object:
Calling init...

Get the value of c.speed:
Calling get...
50

Set the value of c.speed:
Calling set...


WriteSpeedError: speed attribute is read-only

---

## Write-Only Attributes

In [21]:
import hashlib
import os

class User:
    def __init__(self, username, password):
        self.username = username
        self.password = password

    @property
    def password(self):
        raise AttributeError("Password is write-only")

    @password.setter
    def password(self, plaintext):
        salt = os.urandom(32)
        self._hashed_password = hashlib.pbkdf2_hmac(
            "sha256", plaintext.encode("utf-8"), salt, 100_000
        )

In [23]:
user = User('khosro', '123456')
print(user.username)
print(user.password)

khosro


AttributeError: Password is write-only

In [24]:
pass1 = user._hashed_password

In [25]:
user.password = 'salam'
pass1 == user._hashed_password

False

---

## Dynamic vs Static Attribute

### Static attribute

In [26]:
class Circle1:
    def __init__(self, r):
        self.r = r
        self.d = r * 2
#---------------------
c1 = Circle1(10)
print(f'r = {c1.r}, d = {c1.d}')
c1.r = 50
print(f'r = {c1.r}, d = {c1.d}')

r = 10, d = 20
r = 50, d = 20


### Dynamic attribute

In [28]:
class Circle2:
    def __init__(self, r):
        self.r = r
        
    @property
    def d(self):
        return self.r * 2
#---------------------
c2 = Circle2(10)
print(f'r = {c2.r}, d = {c2.d}')
c2.r = 50
print(f'r = {c2.r}, d = {c2.d}')
c2.d = 80

r = 10, d = 20
r = 50, d = 100


AttributeError: can't set attribute

---

## Providing Computed Attributes (Cache)

In [10]:
class Rectangle: # Don't update area
    def __init__(self, width, length):
        self.width = width
        self.length = length
        self.area = self.width * self.length
#---------------------------------------
r = Rectangle(10, 30)
print(r.area)
r.width = 45
print(r.area)    

300
300


In [31]:
# Update area when width and length attribute changes.

from time import sleep
class Rectangle3:
    def __init__(self, width, length):
        self.width = width
        self.length = length
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        self._area = None
        self._width = value

    @property
    def length(self):
        return self._length
    
    @length.setter
    def length(self, value):
        self._area = None
        self._length = value
        
    @property
    def area(self):
        if self._area is None:
            sleep(1)
            self._area = self.width * self.length
        return self._area
#---------------------------------------
r3 = Rectangle3(10, 30)
print(r3.area)
print(r3.area)
print(r3.area)
r3.width = 40
print(r3.area)
print(r3.area)   
print(r3.area)   
r3.length = 800
print(r3.area)
print(r3.area)   
print(r3.area)  

300
300
300
1200
1200
1200
32000
32000
32000


---

## Property and Inheritance

The idea is that if you ever need to override a property in a subclass, then you should provide all the functionality you need in the new version of the property at hand.

In [40]:
class Person:
    def __init__(self, name):
        self._name = name

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value
#-----------------------------------
class Employee(Person):
    pass
    @property
    def name(self):
        return super().name.upper()
    
    ## Notice that you must write setter if you need,
    ## Because you have lost the functionality of setter.

In [45]:
person = Person("John")
print(person.name)
person.name = "John Doe"
print(person.name)
employee = Employee("John")
print(employee.name)
employee.name = "new name"

John
John Doe
JOHN


AttributeError: can't set attribute